## Estraçaõ e Preparação de dados

In [3]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.cm as cm
import seaborn as sns
from Read_Scenarios import *
import os

# ************************************************************************************************
#                                       OPÇÕES DE EJECUÇÃO
# ************************************************************************************************

genscriptbool = False
extract_fromcsv = True

# Analises
ConvergenceAnalise = True
LinhaAnalise = True
ReservaAnalise = True
IntercambiosAnalise = True
#------------------------------
linhascsv = False
reservacsv =False
# CorrelationAnalise = True
PlotGeralPotencia = True
MapasAnalise = True
ComputeDPI = True
resumoIndice = True
# Plots
Plot_Tensao_Geral = True  #PERMITE SALVAR OS DATAFRAMES DE df_Final_ger e df_Final_nt
plotDPI =  True
PlotDPICriticosPot = True
BP_Criticalbuses = True
PlotIntercambios = True

# PlotAnaliseEstab = False

In [ ]:
# path_folder = 'D:/MPV_(FNS Lim)_RC/'
# ============================= CASOS 2022 ===========================================
# path_folder = 'D:/0 FERV/0 Dados PYTHON/Cenarios HPPA PRE-QUALI/MPV_(FNS Lim)_RC/'

# path_folder = 'D:/0 FERV/0 Dados PYTHON/Cenarios HPPA PRE-QUALI/V1A1F2_REV2_091123/'
# path_folder = 'D:/0 FERV/0 Dados PYTHON/Cenarios HPPA PRE-QUALI/V2A2F2_REV4_081123/'
# path_folder = 'D:/0 FERV/0 Dados PYTHON/Cenarios HPPA PRE-QUALI/V3A3F2_REV2_091123/'

# ============================= CASOS 2026 V2A2F_===========================================
# path_folder = 'D:/0 FERV/0 Dados PYTHON/CASOS 2026/V2A2F_/V2A2F2_RESP_FNS_lim_rev2_2026/'
# path_folder = 'D:/0 FERV/0 Dados PYTHON/CASOS 2026/V2A2F_/V2A2F3_RESP_FNS_lim_rev1_2026/'
# path_folder = 'D:/0 FERV/0 Dados PYTHON/CASOS 2026/V2A2F_/V2A2F4_RESP_FNS_lim_rev1_2026/'
# path_folder = 'D:/0 FERV/0 Dados PYTHON/CASOS 2026/V2A2F_/V2A2F5_RESP_FNS_lim_rev1_2026/'
# ============================= CASOS 2026 V1A1F_===========================================
# path_folder = 'D:/0 FERV/0 Dados PYTHON/CASOS 2026/V1A1F_/V1A1F2_RESP_FNS_lim_rev1_2026/'
# path_folder = 'D:/0 FERV/0 Dados PYTHON/CASOS 2026/V1A1F_/V1A1F3_RESP_FNS_lim_rev1_2026/'
# path_folder = 'D:/0 FERV/0 Dados PYTHON/CASOS 2026/V1A1F_/V1A1F4_RESP_FNS_lim_rev1_2026/'
path_folder = 'D:/0 FERV/0 Dados PYTHON/CASOS 2026/V1A1F_/V1A1F5_RESP_FNS_lim_rev1_2026/'

if extract_fromcsv:
    pathcsv1 = path_folder + 'ProcessedDataBase.csv'
    pathcsv2 = None
else:
    pathcsv1 = None
    pathcsv2 = path_folder + 'ProcessedDataBase.csv'

cases = Read_Scenarios(path_folder, RST=False, pathcsv = pathcsv1, genscript=genscriptbool)
cenario = cases.cenario

In [ ]:
if ConvergenceAnalise:
    OPFs_concatenados = cases.OPF
    OPF_NC = OPFs_concatenados[(OPFs_concatenados['Valor4'] == 'F')].copy()
    PWF_NC = OPFs_concatenados[(OPFs_concatenados['Valor5'] == 'F')].copy()
    PWF_CV = OPFs_concatenados[(OPFs_concatenados['Valor5'] == 'T')].copy()
    print('====================================================================')
    print('Numero de casos não Convergidos no PWF: ' + str(len(PWF_NC)) + '=> ' +  str(round(len(PWF_NC)/1344*100,2)))
    print('Numero de casos não Convergidos no OPF: ' + str(len(OPF_NC)) + '=> ' + str(round(len(OPF_NC)/1344*100,2)))
    print('====================================================================')
    ## ====================================================================================================================================================================================
    OPF_NC[['Dia','Hora']].to_csv(cenario+'/OPF_NC.csv', index=None)
    PWF_NC[['Dia','Hora']].to_csv(cenario+'/PWF_NC.csv', index=None)
    print('Os pontos de operação não convergidos no PWF são:')
    bool_PWF_NConv = PWF_NC[['Dia', 'Hora']].apply(tuple, axis=1)
    print(bool_PWF_NConv.values)

In [ ]:
from ProcessData import *

busdata = True
## ***************** (Este código adiciona as barras pertencentes a cada estado e agrupa por região e barra) *****************
print('******************** PREPARAÇÃO DE DADOS ********************')
processdata = ProcessData(cases.DfAnalysis, pathcsv2, extract_fromcsv, busdata)
df_Final_ger = processdata.df_Final_ger
df_Final_nt = processdata.df_Final_nt
dff_Ger_map = processdata.dff_Ger_map
dff_Ger_map.loc[dff_Ger_map['Gen_Type']=='UNE','Gen_Type'] = 'UTE' # cambia de designación de usinas nucleares a termicas para ser plotadas en el mapa
dff_NT_map = processdata.dff_NT_map

DF_REGIONAL_GER = processdata.DF_REGIONAL_GER
DF_REGIONAL_GER['QG/QL'] = DF_REGIONAL_GER['QG_MVAR']/DF_REGIONAL_GER['QL_MVAR']
DF_REGIONAL_GER['PG/PL'] = DF_REGIONAL_GER['PG_MW']/DF_REGIONAL_GER['PL_MW']
DF_REGIONAL_GER['PG_FERV'] =  (DF_REGIONAL_GER['PG_EOL'] + DF_REGIONAL_GER['PG_SOL'])/DF_REGIONAL_GER['PL_MW']
DF_REGIONAL_GER['ReservaINDshunt'] = DF_REGIONAL_GER['SHUNT_INST_IND'] - DF_REGIONAL_GER['Shunt_Ind']
DF_REGIONAL_GER['ReservaCAPshunt'] = DF_REGIONAL_GER['SHUNT_INST_CAP'] - DF_REGIONAL_GER['Shunt_Cap']

DF_REGIONAL_GER[['PG_MW', 'QG_MVAR', 'PL_MW', 'QL_MVAR','Shunt_Ind', 'Shunt_Cap','SHUNT_INST_IND', 'SHUNT_INST_CAP', 'ReservaIND', 'ReservaCAP',
                'PG_UHE', 'PG_UTE', 'PG_EOL', 'PG_SOL', 'PG_BIO', 'PG_Dist', 'QG/QL', 'PG/PL', 'PG_FERV', 'ReservaINDshunt', 'ReservaCAPshunt']].to_csv(cenario+'/DF_POT_Reg.csv')

DF_REGIONAL_PQ = processdata.DF_REGIONAL_PQ

# barras  = ['MSULD3-EOL22', 'CLEMNTEOL-66', 'CLEMNTEOL-60', 'MSULD1-EOL27', 'MSULD2-EOL27', 'MSULD4-EOL08'] #em algun momento asignar EOL no Tipo de usina nessas barras que não esta asignando automaticamente

In [ ]:
## ***************** (Este código obtem as informações das linhas AC e DC e reserva por maquina) *****************
# cases.generatescript()
if linhascsv | reservacsv:
        PWF16_concatenados = dd.read_csv(path_folder + '/LinhasInfo.csv', sep=',').compute()
        PWF16_concatenados['Dia'] = PWF16_concatenados['Dia'].astype(str)
        PWF16_concatenados['Dia'] = PWF16_concatenados['Dia'].str.zfill(2)

        if IntercambiosAnalise:
                cases.get_Intercambios(df = PWF16_concatenados)
        if reservacsv:
                SGN01_concatenados = dd.read_csv(path_folder + '/ReservaInfo.csv', sep=',').compute()
                SGN01_concatenados['Dia'] = SGN01_concatenados['Dia'].astype(str)
                SGN01_concatenados['Dia'] = SGN01_concatenados['Dia'].str.zfill(2)

elif (LinhaAnalise == True )|(ReservaAnalise == True):
    cases.get_Networkinfo(linhas = LinhaAnalise, Reserva = ReservaAnalise, Intercambios = IntercambiosAnalise)
    if LinhaAnalise:
        PWF16_concatenados = cases.linesInfo
        if IntercambiosAnalise:
            DCLinks_concatenados = cases.HVDCInfo
            DF_Intercambios = cases.DF_Intercambios
    if ReservaAnalise:
        SGN01_concatenados = cases.ReserveInfo


# Definição Funcões Gerais

In [ ]:
## Definição de Funções Gerais =========================================================================================

plt.rcParams["font.family"] = "Times New Roman"
paletadcolor = sns.color_palette()

def plot_boxplot(data, labels, title, xlabel, ylabel, limites=None, vert = True, rotation = 0, text = True, nbarra = None):
    
    markerfacecolor = dict(markerfacecolor='gray', marker='o')  # El diccionario que define el color y marcador
    fig1, axs = plt.subplots(figsize=(25, 20))
    if vert:
        axs.boxplot(data, flierprops=markerfacecolor)
        
        # axs.spines['top'].set_visible(False)
        # axs.spines['right'].set_visible(False)
        # axs.spines['left'].set_visible(False)
        # axs.spines['bottom'].set_color('#DDDDDD')
        axs.tick_params(bottom=False, left=False)
        axs.set_axisbelow(True)
        

        # Agregar etiquetas de texto con el número de muestras
        if text:
            for i, data_item in enumerate(data):
                num_muestras = len(data_item)
                if num_muestras > 0: 

                    try:
                        axs.text(i+1, np.max(data_item)+0.005, f'Buses = {nbarra[i].size} ', ha='center', va='bottom',size=18)
                        axs.text(i+1, np.median(data_item), f' {np.mean(data_item):.3f} ', ha='center', va='bottom',size=15)
                        axs.text(i+1.4, np.quantile(data_item, q=0.25), f' {np.quantile(data_item, q=0.25):.3f} ', ha='center', va='bottom',size=15)
                        axs.text(i+1.4, np.quantile(data_item, q=0.75), f' {np.quantile(data_item, q=0.75):.3f} ', ha='center', va='bottom',size=15)
                        
                    except:
                        axs.text(i+1.4, np.mean(data_item), f'{num_muestras}', ha='center', va='bottom',size=25)
        
        if limites != None:
            axs.set_ylim(limites)
            # axs.set_yticks(np.linspace(limites[0],limites[1],12))
        plt.xticks(range(1, len(labels)+1), labels, fontsize=15)

    else:
        axs.boxplot(data, vert=False, flierprops=markerfacecolor)
        if limites != None:
            axs.set_xlim(limites)
            axs.set_xticks(np.linspace(limites[0],limites[1],20))
        if text:
                axs.text(np.median(data), 1.1 , f' {np.mean(data):.3f} ', ha='center', va='bottom',size=18)

    plt.xlabel(xlabel, fontsize=22)
    plt.ylabel(ylabel, fontsize=22)
    plt.title(title, fontsize=25)
    plt.xticks(fontsize=20, rotation = rotation)
    plt.yticks(fontsize=25)
    plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.6)
    axs.xaxis.grid(False)
    nome = cenario + '/BoxPlot/' + title + '.png'
    plt.savefig(nome, bbox_inches = 'tight')
    # nome = cenario + '/BoxPlot/' + title + '.svg'
    # plt.savefig(nome)
    plt.show()

def plot_Potencia(df_data, eje_y, title, limites=None):
    
    fig, axs = plt.subplots(nrows=1, figsize=(20, 10), sharex=False)

    axs.plot(df_data.values, color=sns.color_palette("Paired")[1])
        
    # axs.legend(loc='best', fontsize=12)
    # Calculate the number of data points in a day (assuming each day has 48 data points)
    data_points_per_day = 48
    # Calculate the number of days based on the length of the data
    num_days = len(df_data) // data_points_per_day
    # Set x-axis ticks and labels for each day
    axs.set_xticks([i * data_points_per_day for i in range(num_days)])
    axs.set_xticklabels([f'Day {i+1}' for i in range(num_days)], fontsize=18, rotation=45, ha='right')
    axs.tick_params(axis='y', labelsize=18)
    axs.set_xlabel('Days', fontsize=18)
    axs.set_ylabel(eje_y, fontsize=18)
    axs.set_title(title, fontsize=20)
    if limites != None:
        axs.set_xlim(limites)
    axs.grid()
    plt.tight_layout()
    nome = cenario + '/Potencia/' + title + '.png'
    plt.savefig(nome, bbox_inches = 'tight')
    plt.show()

def plot_reserva_reg (df_data, eje_y, name, title, INDICE, xlimites=None,ylimites=None, display_ = True, order = False):

    if display_:
        display(df_data[df_data[INDICE]==df_data[INDICE].max()].T)
    
    fig, axs = plt.subplots(nrows=1, figsize=(20, 10), sharex=False)
    colores = [sns.color_palette("Paired")[1], sns.color_palette("Paired")[3], sns.color_palette("Paired")[4],sns.color_palette("Paired")[7],sns.color_palette("Paired")[9]]
    region_map = {'Nordeste':'Northeast', 'Norte':'North', 'Sudeste-Centro-Oeste':'SE-CW', 'Sul':'South','AC-RO':'AC-RO'}
    for idx, regiao in enumerate(['Norte','Nordeste','Sudeste-Centro-Oeste', 'Sul', 'AC-RO']):
        
        if order:
            data = df_data.loc[:, :, regiao].sort_values(INDICE, ascending=False)[INDICE]
        else:
            data = df_data.loc[:, :, regiao][INDICE]
        
        axs.plot(data.values, color=colores[idx], label=region_map[regiao], lw=2, linestyle='-')
        
    axs.legend(loc='upper right', fontsize=18)
    axs.tick_params(axis='y', labelsize=24)
    axs.tick_params(axis='x', labelsize=24)
    axs.set_xlabel('Pontos de Operação', fontsize=23)
    axs.set_ylabel(eje_y, fontsize=22)
    axs.set_title(title, fontsize=22)
    if xlimites != None:
        axs.set_xlim(xlimites)
    if ylimites != None:
        axs.set_ylim(ylimites)
    axs.grid(True, linestyle='-', linewidth=1.2, alpha=0.4)
    plt.tight_layout()
    nome = cenario + '/Potencia/' + name + '.png'
    plt.savefig(nome)
    plt.show()

def plot_Intercambio (df_AC, df_DC , eje_y, title, COL_AC, COL_DC, Ylimites=None, Xlimites=  None):

    fig, axs = plt.subplots(nrows=1, figsize=(16, 7))
    colores1 = [paletadcolor[0], paletadcolor[1], paletadcolor[2],paletadcolor[3],paletadcolor[4], paletadcolor[5]]
    colores2 = [paletadcolor[5], paletadcolor[4], paletadcolor[3],paletadcolor[2],paletadcolor[1], paletadcolor[0]]

    # DF_REGIONAL_GER.loc[:,:,'Nordeste']['PG_EOL'].plot(figsize=(10, 6),lw=1.5, color = paletadcolor[5])

    for idx, fluxo in enumerate(COL_AC): 
        data_ = df_AC.loc[fluxo]['MW:From-To']
        axs.plot(data_.values, color=colores1[idx], label= fluxo.replace('_',' '), lw=1.4, linestyle='-')
    for idx, fluxo in enumerate(COL_DC): 
        data_ = df_DC.loc[fluxo][' P(MW)']
        axs.plot(data_.values, color=colores2[idx+1], label= fluxo.replace('_',' '), lw=2.2, linestyle='-')

    axs.xaxis.set_major_locator(plt.MaxNLocator(12))
    axs.legend(loc='best', fontsize=14)
    axs.tick_params(axis='y', labelsize=15)
    axs.tick_params(axis='x', labelsize=15)
    axs.set_xlabel('Semihoras', fontsize=15)
    axs.set_ylabel(eje_y, fontsize=15)
    axs.set_title(title, fontsize=20)
    if Ylimites != None:
        axs.set_ylim(Ylimites)
    if Xlimites != None:
        axs.set_xlim(Xlimites)
    axs.grid(True, linestyle='--', linewidth=1, alpha=0.2)
    plt.tight_layout()
    nome = cenario + '/Intercambios/' + title + '.png'
    plt.savefig(nome)
    plt.show()

def plot_indice_0 (df_data, eje_y, name, title, INDICE, xlimites=None,ylimites=None, order = False, ax=None):
    
    if ax is None:
        fig, axs = plt.subplots(nrows=1, figsize=(15, 6), sharex=False)
    else:
        axs = ax

    colores = [sns.color_palette("Paired")[5], sns.color_palette("Paired")[1], sns.color_palette("Paired")[3]]
    if order:
        data = df_data.sort_values(INDICE, ascending=False)[INDICE]
        data_points_per_day = 10
        num_days = (len(df_data))*data_points_per_day / 100
        axs.set_xticks([round(i * num_days) for i in range(data_points_per_day+1)])
        axs.set_xticklabels([f'{i*10}' for i in range(data_points_per_day+1)], fontsize=12, rotation=0, ha='center')
        axs.set_xlabel('Percentage of half hours in a month (%)', fontsize=23)
    else:
        data = df_data[INDICE]
        axs.set_xlabel('Operating Points', fontsize=23)
    
    area_trapezoidal = np.trapz(data.values)/len(data)
    media = np.mean(data.values)
    axs.plot(data.values, color=colores[1], label='Todos os Cenarios', lw=2, linestyle='-')
        
    axs.legend(loc='upper right', fontsize=18)
    axs.tick_params(axis='y', labelsize=24)
    axs.tick_params(axis='x', labelsize=24)
    axs.set_ylabel(eje_y, fontsize=22)
    axs.set_title(f'{title} normalized area/mean: {area_trapezoidal, media}', fontsize=15)
    if xlimites is not None:
        axs.set_xlim(xlimites)
    if ylimites is not None:
        axs.set_ylim(ylimites)
    axs.grid(True, linestyle='-', linewidth=1.2, alpha=0.4)
    plt.tight_layout()
    if ax is None:
        nome = cenario + '/Indice/' + name + '.png'
        plt.savefig(nome, bbox_inches = 'tight')
        plt.show()
    return area_trapezoidal


# Intercambios AC e DC

In [ ]:
# processdata.Df_VF_SF[(processdata.Df_VF_SF['Dia'] == '01') & (processdata.Df_VF_SF['Hora'] == '00-00')][['BUS_ID','BUS_NAME','VBASEKV','REG', 'U_FED']].to_csv('BarrasGeoreferenciadas.csv',index=None)

def addUF_linha(from_bus, to_bus, bus_info_map, vbasekv_map):

    reg1 = bus_info_map.get(from_bus, np.nan)
    Vbase1 = vbasekv_map.get(from_bus, np.nan)
    reg2 = bus_info_map.get(to_bus, np.nan)

    if reg1 == reg2 and reg1 is not np.nan:
        return reg1, Vbase1
    else:
        return np.nan, np.nan

def Main_linha_addREG(PWF16_concatenados):

    Df_VF_SF = processdata.Df_VF_SF
    InfoBarras = Df_VF_SF[(Df_VF_SF['Dia']=='01') & (Df_VF_SF['Hora']=='00-00')][['BUS_ID','BUS_NAME', 'ARE','VBASEKV','REG', 'U_FED', 'Gen_Type','Latitude', 'Longitude']]
    PWF16_concatenados_d1 = PWF16_concatenados[(PWF16_concatenados['Dia'] == '01') & (PWF16_concatenados['Hora'] == '00-00')].groupby(by=['From#','To#']).first().reset_index().copy()

    # Create a dictionary to map 'BUS_ID' to 'REG' and 'VBASEKV'
    bus_info_map = dict(zip(InfoBarras['BUS_ID'], InfoBarras['REG']))
    vbasekv_map = dict(zip(InfoBarras['BUS_ID'], InfoBarras['VBASEKV']))

    # Use the apply function to create 'REG' and 'VBASEKV' columns in PWF16_concatenados_d1
    PWF16_concatenados_d1['REG'], PWF16_concatenados_d1['VBASEKV'] = zip(*PWF16_concatenados_d1.apply(lambda row: addUF_linha(row['From#'], row['To#'], bus_info_map, vbasekv_map), axis=1))

    PWF16_concatenados_R = PWF16_concatenados.merge(PWF16_concatenados_d1[['From#','To#','REG','VBASEKV']], on=['From#','To#'], how='left')
    PWF16_Filt_linhas = PWF16_concatenados_R[(PWF16_concatenados_R['Type'] == ' TL') & ~(PWF16_concatenados_R['REG'].isna())]
    PWF16_Filt_TRAFO = PWF16_concatenados_R[(PWF16_concatenados_R['Type'] == ' TRAFO') & ~(PWF16_concatenados_R['REG'].isna())]

    return PWF16_Filt_linhas, PWF16_Filt_TRAFO

if LinhaAnalise:

    PWF16_Filt_linhas, PWF16_Filt_TRAFO = Main_linha_addREG(PWF16_concatenados)
    PWF16_Filt_linhas[['From#','To#','From Name','To Name','% L1', 'L1(MVA)', 'Mvar:Losses','Dia', 'Hora','REG', 'VBASEKV','MVA', 'MW:From-To', 'MW:To-From','Power Factor:From-To','Power Factor:To-From']].to_csv(cenario+'/Linhas.csv', index=None)
    PWF16_Filt_TRAFO[['From#','To#','From Name','To Name','% L1', 'L1(MVA)', 'Mvar:Losses','Dia', 'Hora','REG', 'VBASEKV','MVA', 'MW:From-To', 'MW:To-From','Power Factor:From-To','Power Factor:To-From']].to_csv(cenario+'/Trafo.csv', index=None)
    PWF16_Filt_grouped = PWF16_Filt_linhas[PWF16_Filt_linhas['VBASEKV'].isin([230, 345, 440, 500, 525, 765])].groupby(by = ['Dia','Hora','REG']).agg({'% L1':'mean', 'Mvar:Losses':'sum'}) 

    if IntercambiosAnalise:
        ## ========================================== ELOS SEPARADOS POR BIPOLOS:
        # pole_mapping = {1: 'Bipolo1', 2: 'Bipolo1', 3: 'Bipolo2', 4: 'Bipolo2'}
        # dfelo1 = DCLinks_concatenados[DCLinks_concatenados['Bus #'] == 85].groupby(by=['Dia', 'Hora', ' Pole #']).agg({' P(MW)': sum, ' Q(Mvar)': sum})
        # dfelo1['Nome Elo'] = 'Elo_FOZ-IBIUNA'
        # dfelo1['Bipole'] = dfelo1.index.get_level_values(' Pole #').map(pole_mapping)
        # dfelo2 = DCLinks_concatenados[DCLinks_concatenados['Bus #'] == 7055].groupby(by=['Dia', 'Hora', ' Pole #']).agg({' P(MW)': sum, ' Q(Mvar)': sum})
        # dfelo2['Nome Elo'] = 'Elo_PVEL-ARARQ'
        # dfelo2['Bipole'] = dfelo2.index.get_level_values(' Pole #').map(pole_mapping)
        # dfelo3 = DCLinks_concatenados[DCLinks_concatenados['Bus #'] == 7059].groupby(by=['Dia', 'Hora', ' Pole #']).agg({' P(MW)': sum, ' Q(Mvar)': sum})
        # dfelo3['Nome Elo'] = 'Elo_CPVBTB-PVEL'
        # dfelo3['Bipole'] = dfelo3.index.get_level_values(' Pole #').map(pole_mapping)
        # dfelo4 = DCLinks_concatenados[(DCLinks_concatenados['Bus #'] == 8100)].groupby(by=['Dia', 'Hora', ' Pole #']).agg({' P(MW)': sum, ' Q(Mvar)': sum})
        # dfelo4['Nome Elo'] = 'Elo_XINGU-SE'
        # dfelo4['Bipole'] = dfelo4.index.get_level_values(' Pole #').map(pole_mapping)
        # dfelo1.reset_index().groupby(['Dia', 'Hora', 'Bipole']).agg({' P(MW)': sum, 'Nome Elo': 'first'}).to_csv('HVDC_FOZ_IBIUNA.csv')
        # dfelo2.reset_index().groupby(['Dia', 'Hora', 'Bipole']).agg({' P(MW)': sum, 'Nome Elo': 'first'}).to_csv('HVDC_PVEL-ARARQ.csv')

        ## ========================================== ELOS HVDC SEM SEPARAÇÃO POR POLOS:

        dfelo1 = DCLinks_concatenados[DCLinks_concatenados['Bus #'] == 85].groupby(by=['Dia', 'Hora']).agg({' P(MW)': 'sum', ' Q(Mvar)': 'sum'})
        dfelo1['Nome Elo'] = 'Elo_FOZ-IBIUNA'
        dfelo2 = DCLinks_concatenados[DCLinks_concatenados['Bus #'] == 7055].groupby(by=['Dia', 'Hora']).agg({' P(MW)': 'sum', ' Q(Mvar)': 'sum'})
        dfelo2['Nome Elo'] = 'Elo_PVEL-ARARQ'
        dfelo3 = DCLinks_concatenados[DCLinks_concatenados['Bus #'] == 7059].groupby(by=['Dia', 'Hora']).agg({' P(MW)': 'sum', ' Q(Mvar)': 'sum'})
        dfelo3['Nome Elo'] = 'Elo_CPVBTB-PVEL'
        dfelo4 = DCLinks_concatenados[(DCLinks_concatenados['Bus #'] == 8100)].groupby(by=['Dia', 'Hora']).agg({' P(MW)': 'sum', ' Q(Mvar)': 'sum'})
        dfelo4['Nome Elo'] = 'Elo_XINGU-SE'
        dfelo4 = DCLinks_concatenados[(DCLinks_concatenados['Bus #'] == 8100) & (DCLinks_concatenados[' Pole #'].isin([1,2]))].groupby(by=['Dia', 'Hora']).agg({' P(MW)': 'sum', ' Q(Mvar)': 'sum'})
        # dfelo4['Nome Elo'] = 'Elo_XINGU-ESTREI'
        # dfelo5 = DCLinks_concatenados[(DCLinks_concatenados['Bus #'] == 8100) & (DCLinks_concatenados[' Pole #'].isin([3,4]))].groupby(by=['Dia', 'Hora']).agg({' P(MW)': 'sum', ' Q(Mvar)': 'sum'})
        # dfelo5['Nome Elo'] = 'Elo_XINGU-T.RIO'
        # Merge all dataframes
        # df_HVDC = pd.concat([dfelo1, dfelo2, dfelo3, dfelo4, dfelo5], axis=0, keys=['Elo_FOZ-IBIUNA', 'Elo_PVEL-ARARQ', 'Elo_CPVBTB-PVEL' ,'Elo_XINGU-ESTREI', 'Elo_XINGU-T.RIO'])
        df_HVDC = pd.concat([dfelo1, dfelo2, dfelo3, dfelo4], axis=0, keys=['Elo_FOZ-IBIUNA', 'Elo_PVEL-ARARQ', 'Elo_CPVBTB-PVEL' ,'Elo_XINGU-SE'])
        df_HVDC.to_csv(cenario+'/DF_HVDC.csv')

        if PlotIntercambios == True:
            plot_Intercambio (DF_Intercambios, df_HVDC , '(MW)', 'Exportação (N-S, NE-SE) e  Elo Xingu-SE', ['Fluxo_N-S', 'Fluxo_NE-SE'], ['Elo_XINGU-SE'], )
            plot_Intercambio (DF_Intercambios, df_HVDC , '(MW)', 'Exportação N-S e  Elo Xingu-SE', ['Fluxo_N-S'], ['Elo_XINGU-SE'], )
            plot_Intercambio (DF_Intercambios, df_HVDC , '(MW)', 'Comparativo Exportação NE-N e Elo Xingu-SE', ['Fluxo_NE-N'], ['Elo_XINGU-SE'], Xlimites=None)
            plot_Intercambio (DF_Intercambios, df_HVDC , '(MW)', 'Comparativo Exportação NE-SE e Elo FOZ-IBIUNA', ['Fluxo_NE-SE'], ['Elo_FOZ-IBIUNA'], Xlimites=None)
            plot_Intercambio (DF_Intercambios, df_HVDC , '(MW)', 'Comparativo Exportação SUL-SECO e Elo FOZ-IBIUNA', ['Fluxo_SUL-SECO'], ['Elo_FOZ-IBIUNA'], Xlimites=None)


# Reserva Regional

In [ ]:
if ReservaAnalise == True:
    if (SGN01_concatenados.empty == False):

        df_Final_ger_mod = df_Final_ger[(df_Final_ger['Dia'] == '01') & (df_Final_ger['Hora'] == '00-00')][['BUS_ID', 'Gen_Type', 'U_FED', 'REG']]
        SGN01_concatenados.rename(columns={'Bus':'BUS_ID', }, inplace=True)
        Df_Reserva = SGN01_concatenados.merge(df_Final_ger_mod, how = 'left', on='BUS_ID')

        REG_groupReserve = Df_Reserva.groupby(by = ['Dia','Hora', 'REG']).agg({' Reserve': sum})
        GroupReserve = Df_Reserva.groupby(by = ['Dia','Hora']).agg({' Reserve': sum})
        GroupReserve[' Reserve'].to_csv(cenario + '/Reserva_MW_PO.csv', header=True, index=True)
        REG_groupReserve[' Reserve'].to_csv(cenario + '/Reserva_MW_PO_REG.csv', header=True, index=True)

        plot_Potencia(GroupReserve[' Reserve'], '(MW)', 'RESERVA (MW) - SIN', limites=None)
        plot_reserva_reg (REG_groupReserve, '(MW)', 'Reserva por Região', 'RESERVA POR REGIÃO', ' Reserve', xlimites=None,ylimites=None, display_ = False, order = False)

# =====================================ESSE DATAFRAME É SÓ DA RESERVA DAS MAQUINAS COM MODELO DO GERADOR PARA O CONTROLE DE FREQ
# ===========================================================================================================================

    dff_reserva = Df_Reserva.merge(df_Final_ger[['BUS_ID','Dia', 'Hora', 'QMX_MVAR', 'QMN_MVAR', 'Ger_Units','QG_MVAR']], on=['BUS_ID','Dia', 'Hora'], how='left')
        # Calculate 'Qmin' and 'Qmax' using vectorized operations
    dff_reserva['Qmin'] = (dff_reserva['QMN_MVAR'] / dff_reserva['Ger_Units']) * dff_reserva[' Units']
    dff_reserva['Qmax'] = (dff_reserva['QMX_MVAR'] / dff_reserva['Ger_Units']) * dff_reserva[' Units']
    # Use numpy's where function for conditional assignment in a vectorized way
    dff_reserva['ReservaIND'] = np.where(dff_reserva['QG_MVAR'] < 0, dff_reserva['Qmin'] - dff_reserva['QG_MVAR'], dff_reserva['Qmin'])
    dff_reserva['ReservaCAP'] = np.where(dff_reserva['QG_MVAR'] > 0, dff_reserva['Qmax'] - dff_reserva['QG_MVAR'], dff_reserva['Qmax'])
# ============================================================================================================================

    dffreservaPO = df_Final_ger.groupby(['Dia', 'Hora']).agg({'QG_MVAR': sum, 'ReservaIND':sum, 'ReservaCAP':sum})
    dffreservaPO_REG = df_Final_ger.groupby(['Dia', 'Hora', 'REG']).agg({'QG_MVAR': sum, 'ReservaIND':sum, 'ReservaCAP':sum})
    # Salvando dataframe de reserva mvar ============================================
    dffreservaPO.to_csv(cenario + '/ReservaMVAR_PO.csv', header=True, index=True)
    dffreservaPO_REG.to_csv(cenario + '/ReservaMVAR_PO_REG.csv', header=True, index=True)

#=============================================================================================================================
#                                                                   PLOTS RESERVA MVAR
#=============================================================================================================================

    plot_reserva_reg (dffreservaPO_REG, '(MVAR)', 'Reserva Capacitiva por Região MVAR', 'RESERVA CAPACITIVA POR REGIÃO MVAR', 'ReservaCAP', xlimites=None,ylimites=None, display_ = False, order = False)
    plot_reserva_reg (dffreservaPO_REG, '(MVAR)', 'Reserva Indutiva por Região MVAR', 'RESERVA INDUTIVA POR REGIÃO MVAR', 'ReservaIND', xlimites=None,ylimites=None, display_ = False, order = False)

    fig, ax = plt.subplots(figsize=(20,10))
    dffreservaPO['ReservaCAP'].plot(figsize=(20,10), grid=True, title='RESERVA CAPACITIVA (Mvar)',legend='RESERVA')
    ax.tick_params(axis='x', labelsize=15)
    ax.tick_params(axis='y', labelsize=15)
    ax.set_xlabel('PO',fontsize = 15)
    ax.set_ylabel('(MVAR)',fontsize = 15)
    ax.set_title('RESERVA CAPACITIVA (Mvar)', fontsize = 20)
    ax.legend(fontsize = 15)
    nome = cenario + '/Potencia/Reserva_cap_mvar.png'
    plt.savefig(nome, bbox_inches = 'tight')

    fig, ax = plt.subplots(figsize=(20,10))
    dffreservaPO['ReservaIND'].plot(figsize=(20,10), grid=True, title='RESERVA INDUTIVA (Mvar)',legend='RESERVA')
    ax.tick_params(axis='x', labelsize=15)
    ax.tick_params(axis='y', labelsize=15)
    ax.set_xlabel('PO',fontsize = 15)
    ax.set_ylabel('(MVAR)',fontsize = 15)
    ax.set_title('RESERVA INDUTIVA (Mvar)', fontsize = 20)
    ax.legend(fontsize = 15)
    nome = cenario + '/Potencia/Reserva_ind_mvar.png'
    plt.savefig(nome, bbox_inches = 'tight')

# Potencia e Tensão - Geral

### a) Potencia Ativa e Reativa

In [ ]:
regioes = DF_REGIONAL_GER.reset_index()['REG'].unique()
df_pg = DF_REGIONAL_GER.reset_index(level=['Dia','Hora', 'REG'])[['PG_MW','PL_MW','PG_EOL','PG_SOL', 'Dia', 'Hora', 'MODV_PU', 'QG_MVAR']]
df_pg['PG_FERV'] =  (df_pg['PG_EOL'] + df_pg['PG_SOL'])/df_pg['PL_MW']
df_grouped = df_pg.groupby(by = ['Dia', 'Hora'])[['PG_FERV', 'MODV_PU', 'QG_MVAR','PG_MW', 'PL_MW' ]].sum()
 # Salvando dataframe de POTENCIA MW, RESERVA E MVAR===========================================
df_grouped['PG_MW'].to_csv(cenario + '/MW_PO.csv', header=True, index=True)
df_grouped['QG_MVAR'].to_csv(cenario + '/MVAR_PO.csv', header=True, index=True)
# =============================================================================================
if PlotGeralPotencia:
    plot_Potencia(df_grouped['QG_MVAR'], '(MVAR)', 'POTENCIA REATIVA GERADA (MVAR) - SIN', limites=None)
    plot_Potencia(df_grouped['PG_MW'], '(MW)', 'POTENCIA ATIVA GERADA (MW) - SIN', limites=None)
    typeGenDic = {'QG_UHE':'Num_Usinas_UHE', 'QG_UTE':'Num_Usinas_UTE', 'QG_EOL':'Num_Usinas_EOL','QG_SOL':'Num_Usinas_SOL', 'QG_BIO':'Num_Usinas_BIO'}
    typeGenRegDic = {'Norte':['QG_UHE','QG_EOL','QG_SOL','QG_UTE'],'Nordeste':['QG_UHE','QG_EOL','QG_SOL','QG_UTE'],'Sudeste-Centro-Oeste':['QG_UHE','QG_EOL','QG_SOL','QG_UTE','QG_BIO'],'Sul':['QG_UHE','QG_EOL','QG_UTE','QG_BIO'], 'AC-RO':['QG_UHE','QG_UTE']}
    typeGenRegDic_MW = {'Norte':['PG_UHE','PG_EOL','PG_SOL','PG_UTE'],'Nordeste':['PG_UHE','PG_EOL','PG_SOL','PG_UTE'],'Sudeste-Centro-Oeste':['PG_UHE','PG_EOL','PG_SOL','PG_UTE','PG_BIO'],'Sul':['PG_UHE','PG_EOL','PG_UTE','PG_BIO'], 'AC-RO':['PG_UHE','PG_UTE']}
    typeGenDic_MW = {'PG_UHE':'Num_Usinas_UHE', 'PG_UTE':'Num_Usinas_UTE', 'PG_EOL':'Num_Usinas_EOL','PG_SOL':'Num_Usinas_SOL', 'PG_BIO':'Num_Usinas_BIO'}

    for reg in regioes:
        plot_Potencia(DF_REGIONAL_GER.loc[:,:,reg]['QG_MVAR'], '(MVAR)', 'POTENCIA REATIVA GERADA (MVAR) - ' + reg, limites=None)
        plot_Potencia(DF_REGIONAL_GER.loc[:,:,reg]['PG_MW'], '(MW)', 'POTENCIA ATIVA GERADA (MW) - ' + reg, limites=None)
        for tog in typeGenRegDic[reg]:
            numUsinas = DF_REGIONAL_GER.loc[:,:,reg][typeGenDic[tog]][0]
            nome = str('MVAR ' + reg.replace('-',' ')  + ' (' + tog.replace('_','-') + ') - Numero de Usinas ' + str(int(numUsinas)))
            plot_Potencia(DF_REGIONAL_GER.loc[:,:,reg][tog], '(MVAR)', nome , limites=None)

        for tog in typeGenRegDic_MW[reg]:
            numUsinas = DF_REGIONAL_GER.loc[:,:,reg][typeGenDic_MW[tog]][0]
            nome = str('MW ' + reg.replace('-',' ')  + ' (' + tog.replace('_','-') + ') - Numero de Usinas ' + str(int(numUsinas)))
            plot_Potencia(DF_REGIONAL_GER.loc[:,:,reg][tog], '(MW)', nome , limites=None)

### b) Tensão no SIN boxplot

In [ ]:
def boxplot_barrasGeracao(Df_VF):
    
    Df_groupbyUF = Df_VF.groupby(by = ['Gen_Type']).agg({'BUS_ID': 'unique', 'MODV_PU': list,})
    Df_groupbyReg = Df_VF.groupby(by = ['REG']).agg({'BUS_ID': 'unique', 'MODV_PU': list,})
    #***************************************************************************************************
    data =  Df_groupbyUF['MODV_PU'][['UHE', 'UTE', 'PCH', 'EOL', 'UFV', 'BIO']] 
    Nbarras = Df_groupbyUF['BUS_ID'][['UHE', 'UTE', 'PCH', 'EOL', 'UFV', 'BIO']]
    labels = ['Hydro','Thermal','SHP', 'Wind', 'Solar', 'Bio']  # Tus etiquetas de datos
    title = 'Bus Voltage Distribution by Type of Generation'  # Tu título
    xlabel = 'Type of Generation'  # Tu etiqueta del eje x
    ylabel = 'Voltage (pu)'  # Tu etiqueta del eje y
    # plot_boxplot(data, labels, title, xlabel, ylabel,text= True, nbarra = Nbarras)#limites = [0.88,1.125]
    #***************************************************************************************************
    Nbarras = Df_groupbyReg['BUS_ID'][['Norte','Nordeste','Sudeste-Centro-Oeste','Sul','AC-RO']]
    data = Df_groupbyReg['MODV_PU'][['Norte','Nordeste','Sudeste-Centro-Oeste','Sul','AC-RO']]
    labels = ['North', 'Northeast', 'SE-CW', 'South','AC-RO']  # Tus etiquetas de datos
    title = 'Bus Voltage Distribution by Region for Voltage-Controled Buses'  # Tu título
    xlabel = 'Region'  # Tu etiqueta del eje x
    ylabel = 'Voltage (pu)'  # Tu etiqueta del eje y
    # plot_boxplot(data, labels, title, xlabel, ylabel,text= True, nbarra = Nbarras)#, limites = [0.82,1.18]

def boxplot_barrasCarga(Df_Vfpt):

    #***************************************************************************************************
    V_NT = Df_Vfpt.groupby(by = ['VBASEKV']).agg({'BUS_ID': 'unique', 'MODV_PU': list,})
    n_bar = V_NT['BUS_ID'][[138,230,345,440,500,525,765]]
    numbarra = list(n_bar)
    data = V_NT['MODV_PU'][[138,230,345,440,500,525,765]]
    labels = ['138','230','345','440','500','525','765']  # Tus etiquetas de datos
    title = 'Bus Voltage Distribution by Voltage Level'  # Tu título
    xlabel = 'Voltage Level (kV)'  # Tu etiqueta del eje x
    ylabel = 'Voltage (pu)'  # Tu etiqueta del eje y
    # plot_boxplot(data, labels, title, xlabel, ylabel, text=True, nbarra = numbarra)#limites = [0.95, 1.105]
    #***************************************************************************************************
    Df_groupbyReg = Df_Vfpt.groupby(by = ['REG']).agg({'BUS_ID': 'unique', 'MODV_PU': list,})
    Nbarras = Df_groupbyReg['BUS_ID'][['Norte','Nordeste','Sudeste-Centro-Oeste','Sul','AC-RO']]
    data = Df_groupbyReg['MODV_PU'][['Norte','Nordeste','Sudeste-Centro-Oeste','Sul','AC-RO']]
    labels =  ['North', 'Northeast', 'SE-CW', 'South','AC-RO']  # Tus etiquetas de datos
    title = 'Bus Voltage Distribution by Region for Load Buses'  # Tu título
    xlabel = 'Region'  # Tu etiqueta del eje x
    ylabel = 'Voltage (pu)'  # Tu etiqueta del eje y
    # plot_boxplot(data, labels, title, xlabel, ylabel,text= True, nbarra = Nbarras)#, limites = [0.82,1.18]

def plottensaoG():
    Df_VF = processdata.Df_VF
    DFF_Geral = Df_VF[(Df_VF['VBASEKV'].isin([138,230,345,440,500,525,765])) | (Df_VF['Gen_Type'].isin(['UHE', 'UTE', 'PCH', 'EOL', 'UFV', 'BIO']))] 
    filtro1 = (DFF_Geral[['Dia', 'Hora']].apply(tuple, axis=1).isin(bool_PWF_NConv))
    DFF_Geral_PWFC = DFF_Geral[~filtro1]

    # DFF_Geral_PWFC['MODV_PU'].to_csv(cenario+'/Voltage.csv', index=None)

    data = [DFF_Geral_PWFC['MODV_PU'].values]  # Tu conjunto de datos
    labels = ['G. Sincrona']  # Tus etiquetas de datos
    title = 'Bus Voltage Distribution of the System'  # Tu título
    xlabel = 'Voltage (pu)'  # Tu etiqueta del eje x
    ylabel = 'Bus Voltages'  # Tu etiqueta del eje y

    # plot_boxplot(data, labels, title, xlabel, ylabel, vert = False, text=True, rotation = 0)

    # Q1 = np.percentile(data, 25)
    # Q3 = np.percentile(data, 75)
    # print(f'Primer cuartil (Q1): {Q1}')
    # print(f'Tercer cuartil (Q3): {Q3}')
    # IQR = Q3 - Q1
    # limite_inferior = Q1 - 1.5 * IQR
    # limite_superior = Q3 + 1.5 * IQR
    # print(f'El límite inferior es: {limite_inferior}')
    # print(f'El límite superior es: {limite_superior}')

def plottensaoPR():
    df_ger = df_Final_ger[df_Final_ger['Gen_Type'].isin(['UHE', 'UTE', 'PCH', 'EOL', 'UFV', 'BIO'])]
    filtro1 = (df_ger[['Dia', 'Hora']].apply(tuple, axis=1).isin(bool_PWF_NConv))
    df_Final_ger_PWFC = df_ger[~filtro1]
    df_Final_ger_PWFC[['BUS_ID','ARE', 'MODV_PU', 'ANGV_DEG','PG_MW', 'QG_MVAR','Dia', 'Hora', 'U_FED', 'Gen_Type', 'REG', 'B0_MVAR', 'ST', 'SHUNT_INST_IND', 'SHUNT_INST_CAP', 'ReservaIND', 'ReservaCAP']].to_csv(cenario+'/Df_ger.csv', index=None)

    df_nt = df_Final_nt[df_Final_nt['VBASEKV'].isin([138,230,345,440,500,525,765])]
    filtro2 = (df_nt[['Dia', 'Hora']].apply(tuple, axis=1).isin(bool_PWF_NConv))
    df_Final_nt_PWFC = df_nt[~filtro2]
    df_Final_nt_PWFC[['BUS_ID','ARE', 'MODV_PU', 'ANGV_DEG','VBASEKV', 'PL_MW', 'QL_MVAR','Dia', 'Hora', 'U_FED', 'REG', 'B0_MVAR', 'ST', 'SHUNT_INST_IND', 'SHUNT_INST_CAP','ReservaINDshunt', 'ReservaCAPshunt']].to_csv(cenario+'/Df_nt.csv', index=None)
    
    # boxplot_barrasCarga(df_Final_nt_PWFC)
    # boxplot_barrasGeracao(df_Final_ger_PWFC)

if ConvergenceAnalise:
    if Plot_Tensao_Geral:
        plottensaoG()
        plottensaoPR()


# Mapas

In [ ]:
# df = pd.read_csv('RECURSOS/InfoBarras.csv',sep=';')
# df1 = pd.read_csv('C:/Users/David/OneDrive/Desktop/CHRISTOPHER/GeoINFO_BusesSIN.csv', sep= ';')
# # Suponiendo que 'df' es tu DataFrame
# df1 = df1.drop(df1.columns[0], axis=1)
# df2 = df[['BUS_ID','BUS_NAME','Gen_Type','Latitude','Longitude']].merge(df1[['BUS_ID','U_FED','REG']], on='BUS_ID').drop_duplicates()
# df2.to_csv('RECURSOS/GeoINFO_BusesSIN.csv', sep=';')
# processdata.Df_VF.to_csv('Df_VF.csv', index=None)
# dff_NT_map.to_csv('dff_NT_map.csv', index=None)
# dff_Ger_map.to_csv('dff_Ger_map.csv', index=None)

In [ ]:
from Maps import *
if MapasAnalise:
    Df_VF = processdata.Df_VF
    options = {'Limit Violations All': True, 'Mean and Variance': True, 'Limit Violations by Group': True, 'HeatMap by state 1': True, 'Limit Violations PO': False}
    Maps(Df_VF, dff_NT_map, dff_Ger_map, cenario, options)

# STATIC ASSESMENT

## a) Calculo do Indice de tensão

In [ ]:
if ComputeDPI:
    from computeDPI import *

    print('CALCULO DO DPI para todos os Cenários:')
    ts = 0.8
    tb = 1
    VVI = computeDPI(df_Final_nt, df_Final_ger, ts, tb, p_norm = 2, p_inf = False, NBcv = True)
    dfPQ_CSI = VVI.dfPQ_CSI
    dfPV_CSI = VVI.dfPV_CSI
    df_PQ_reg = VVI.df_PQ_reg
    df_PV_reg = VVI.df_PV_reg
    df_busPQ = VVI.df_busPQ
    df_busPV = VVI.df_busPV
    n_maior = VVI.n_maior

    dfPQ_CSI = dfPQ_CSI.groupby(['Dia' , 'Hora', 'REG']).first()
    dfPV_CSI = dfPV_CSI.groupby(['Dia' , 'Hora', 'REG']).first()
    dffPQgb = df_PQ_reg.groupby(by=['Dia','Hora','REG','VBASEKV']).agg({'CSI_INF':'first','CSI_SUP':'first'})
    dffPVgb = df_PV_reg.groupby(by=['Dia','Hora','REG','Gen_Type']).agg({'CSI_INF':'first','CSI_SUP':'first'})

    if ConvergenceAnalise:
        for index in bool_PWF_NConv:
            dfPQ_CSI.drop((index[0], index[1]), inplace=True)
            dfPV_CSI.drop((index[0], index[1]), inplace=True)
            dffPQgb.drop((index[0], index[1]), inplace=True)
            dffPVgb.drop((index[0], index[1]), inplace=True)
        filtro1 = (df_busPQ[['Dia', 'Hora']].apply(tuple, axis=1).isin(bool_PWF_NConv))
        df_busPQ_mod = df_busPQ[~filtro1].copy()
        filtro2 = (df_busPV[['Dia', 'Hora']].apply(tuple, axis=1).isin(bool_PWF_NConv))
        df_busPV_mod = df_busPV[~filtro2].copy()
    else:
        df_busPQ_mod = df_busPQ
        df_busPV_mod = df_busPV
        
    dffPQgb.to_csv(cenario+'/Indice_DecompPQ.csv',index = True)
    dffPVgb.to_csv(cenario+'/Indice_DecompPV.csv',index = True)

## b) Resumo Indice 

In [ ]:
if ComputeDPI:
    n=2

    dfPQ_CSI['DPI_2N_INF'] = dfPQ_CSI['CSI_INF_FINAL'].pow(2*n) 
    dfPQ_CSI['DPI_2N_SUP'] = dfPQ_CSI['CSI_SUP_FINAL'].pow(2*n) 
    dfPV_CSI['DPI_2N_INF'] = dfPV_CSI['CSI_INF_FINAL'].pow(2*n) 
    dfPV_CSI['DPI_2N_SUP'] = dfPV_CSI['CSI_SUP_FINAL'].pow(2*n) 
    ddf_pq = dfPQ_CSI.reset_index().groupby(by = ['Dia','Hora']).agg({'DPI_2N_INF':  'sum', 'DPI_2N_SUP':  'sum'})
    ddf_pq['PQ_lower'] = ddf_pq['DPI_2N_INF'].pow(1/(2*n))
    ddf_pq['PQ_upper'] = ddf_pq['DPI_2N_SUP'].pow(1/(2*n))
    ddf_pv = dfPV_CSI.reset_index().groupby(by = ['Dia','Hora']).agg({'DPI_2N_INF':  'sum', 'DPI_2N_SUP':  'sum'})
    ddf_pv['PV_lower'] = ddf_pv['DPI_2N_INF'].pow(1/(2*n))
    ddf_pv['PV_upper'] = ddf_pv['DPI_2N_SUP'].pow(1/(2*n))
    DF_DPI_pq_pv_ul = pd.concat([ddf_pv[['PV_lower','PV_upper']],ddf_pq[['PQ_lower','PQ_upper']]], axis=1)
    DF_DPI_pq_pv_ul.to_csv(cenario+'/Indice_Modif.csv')

    # ==========================================================================================================================
    dfPQ_CSI['DPI_2N_INF'] = dfPQ_CSI['CSI_INF_FINAL'].pow(2*n) 
    dfPQ_CSI['DPI_2N_SUP'] = dfPQ_CSI['CSI_SUP_FINAL'].pow(2*n) 
    dfPV_CSI['DPI_2N_INF'] = dfPV_CSI['CSI_INF_FINAL'].pow(2*n) 
    dfPV_CSI['DPI_2N_SUP'] = dfPV_CSI['CSI_SUP_FINAL'].pow(2*n) 

    df_DPI_PO_ = dfPQ_CSI['DPI_2N_INF'] + dfPQ_CSI['DPI_2N_SUP'] + dfPV_CSI['DPI_2N_INF'] +  dfPV_CSI['DPI_2N_SUP'] 
    df_DPI_PO = pd.DataFrame(df_DPI_PO_)
    df_DPI_PO = df_DPI_PO.reset_index().groupby(by = ['Dia','Hora']).agg(DPI_PO =(0, 'sum'))
    df_DPI_PO['DPI_PO_final']= df_DPI_PO['DPI_PO'].pow(1/(2*n))
    df_DPI_PO['DPI_PO_final'].to_csv(cenario+'/Indice_PO.csv')

    plot_indice_0 (df_DPI_PO, r'$\mathrm{DPI}$', 'DPI_PO_final','','DPI_PO_final', order=True, ylimites=[-0.05, 1.5] )
    (df_DPI_PO[df_DPI_PO['DPI_PO_final']>1].index.to_frame()[['Dia', 'Hora']].apply(tuple, axis=1)).to_csv(cenario + '/PO_Inseguros.txt', index = None)

In [ ]:
if ComputeDPI:
    if resumoIndice:
        if ConvergenceAnalise:
            path_txt = cenario + '/RelatorioIndiceG.txt'
            numeroPO = len(set(dfPQ_CSI.index.to_frame()[['Dia','Hora']].apply(tuple, axis=1).values))
            DFS_INS = [dfPQ_CSI[dfPQ_CSI['CSI_INF_FINAL']>1],
                    dfPQ_CSI[dfPQ_CSI['CSI_SUP_FINAL']>1],
                    dfPV_CSI[dfPV_CSI['CSI_INF_FINAL']>1],
                    dfPV_CSI[dfPV_CSI['CSI_SUP_FINAL']>1]]
            CasosInseguros = pd.concat(DFS_INS, axis=0,)
            CasosInseguros = CasosInseguros.sort_index()
            lista = [index[:2] for index in CasosInseguros.index]
            conjunto_filtros_INSG = set(map(tuple, lista))
            filtro1 = (OPF_NC[['Dia', 'Hora']].apply(tuple, axis=1).isin(conjunto_filtros_INSG))
            OPF_Filtrado = OPF_NC[filtro1]
            filtro2 = (PWF_CV[['Dia', 'Hora']].apply(tuple, axis=1).isin(conjunto_filtros_INSG))
            PWF_Filtrado = PWF_CV[filtro2]
            print('Numero de casos com indice >  1, INSEGUROS: ' + str(len(conjunto_filtros_INSG)) + '=> ' + str(round(len(conjunto_filtros_INSG)/numeroPO*100,2)))
            print('====================================================================')
            print('Numero de casos com indice >  1 que são não Convergidos no OPF: ' + str(len(OPF_Filtrado)) + '=> ' + str(round(len(OPF_Filtrado)/numeroPO*100,2)) )
            with open(path_txt, 'w') as f:
                f.write('Numero de casos com indice >  1, INSEGUROS:' + str(len(conjunto_filtros_INSG)) + '=> ' + str(round(len(conjunto_filtros_INSG)/numeroPO*100,2)) + '\n')
                f.write('Numero de casos com indice >  1 que são não Convergidos no OPF: ' + str(len(OPF_Filtrado)) + '=> ' + str(round(len(OPF_Filtrado)/numeroPO*100,2)) + '\n' )
            
            DFS_ALARM = [dfPQ_CSI[(dfPQ_CSI['CSI_INF_FINAL']<=1) & (dfPQ_CSI['CSI_INF_FINAL']>0)],
                        dfPQ_CSI[(dfPQ_CSI['CSI_SUP_FINAL']<=1) & (dfPQ_CSI['CSI_SUP_FINAL']>0)],
                        dfPV_CSI[(dfPV_CSI['CSI_INF_FINAL']<=1) & (dfPV_CSI['CSI_INF_FINAL']>0)],
                        dfPV_CSI[(dfPV_CSI['CSI_SUP_FINAL']<=1) & (dfPV_CSI['CSI_SUP_FINAL']>0)]]

            CasosAlarme = pd.concat(DFS_ALARM, axis=0,)
            filtroalarme = (CasosAlarme.index.to_frame()[['Dia', 'Hora']].apply(tuple, axis=1).isin(conjunto_filtros_INSG))
            CasosAlarme = CasosAlarme[~filtroalarme]
            CasosAlarme = CasosAlarme.sort_index()
            lista = [index[:2] for index in CasosAlarme.index]
            conjunto_filtros_ALRM = set(map(tuple, lista))
            filtro1 = (OPF_NC[['Dia', 'Hora']].apply(tuple, axis=1).isin(conjunto_filtros_ALRM))
            OPF_Filtrado = OPF_NC[filtro1]
            filtro2 = (PWF_CV[['Dia', 'Hora']].apply(tuple, axis=1).isin(conjunto_filtros_ALRM))
            PWF_Filtrado = PWF_CV[filtro2]
            print('====================================================================')
            print('Numero de casos com indice  ALARME: ' + str(len(conjunto_filtros_ALRM)) + '=> ' + str(round(len(conjunto_filtros_ALRM)/numeroPO*100,2)))
            print('====================================================================')
            print('Numero de casos com indice ALARME que são não Convergidos no OPF: ' + str(len(OPF_Filtrado)) + '=> ' + str(round(len(OPF_Filtrado)/numeroPO*100,2)))
            with open(path_txt, 'a') as f:
                    f.write('Numero de casos com indice ALARME:' + str(len(conjunto_filtros_ALRM)) + '=> ' + str(round(len(conjunto_filtros_ALRM)/numeroPO*100,2)) + '\n')
                    f.write('Numero de casos com indice ALARME que são não Convergidos no OPF: ' + str(len(OPF_Filtrado)) + '=> ' + str(round(len(OPF_Filtrado)/numeroPO*100,2)) + '\n' )
            
            # ===========================================================================================================================================================
            def print_results(df_CSI, tipo_analise):
                with open(path_txt, 'a') as output_file:
                    output_file.write(f'******** RESULTADOS BARRAS {tipo_analise} INSEGURO ********\n')
                    tipo_sup_inf = 'CSI_SUP_FINAL' if 'SOBRETENSÃO' in tipo_analise else 'CSI_INF_FINAL'
                    
                    for i in df_Final_nt['REG'].unique():
                        indregsuperior = df_CSI[df_CSI[tipo_sup_inf] > 1]
                        try:
                            output_file.write(f'casos com índice acima de 1 na região {i}: {len(set(map(tuple, indregsuperior.loc[:, :, i].index)))}\n')
                        except:
                            output_file.write(f'NÃO TEM casos com índice acima de 1 na região {i}\n')
                            pass

            print_results(dfPQ_CSI, 'BARRAS PQ SOBRETENSÃO')
            print_results(dfPV_CSI, 'BARRAS PV SOBRETENSÃO')
            print_results(dfPQ_CSI, 'BARRAS PQ SUBTENSÃO')
            print_results(dfPV_CSI, 'BARRAS PV SUBTENSÃO')
            
            # ===========================================================================================================================================================
            print('===========================SUBTENSÃO==============================')
            DFS_INS = [dfPQ_CSI[dfPQ_CSI['CSI_INF_FINAL']>1],
                    dfPV_CSI[dfPV_CSI['CSI_INF_FINAL']>1]]
            CasosInseguros = pd.concat(DFS_INS, axis=0,)
            CasosInseguros = CasosInseguros.sort_index()
            lista = [index[:2] for index in CasosInseguros.index]
            conjunto_filtros_INSG = set(map(tuple, lista))
            filtro1 = (OPF_NC[['Dia', 'Hora']].apply(tuple, axis=1).isin(conjunto_filtros_INSG))
            OPF_Filtrado = OPF_NC[filtro1]
            filtro2 = (PWF_CV[['Dia', 'Hora']].apply(tuple, axis=1).isin(conjunto_filtros_INSG))
            PWF_Filtrado = PWF_CV[filtro2]
            print('Numero de casos com indice >  1, INSEGUROS: ' + str(len(conjunto_filtros_INSG)) + '=> ' + str(round(len(conjunto_filtros_INSG)/1344*100,2)))
            print('====================================================================')
            print('Numero de casos com indice >  1 que são não Convergidos no OPF: ' + str(len(OPF_Filtrado)) + '=> ' + str(round(len(OPF_Filtrado)/1344*100,2)) )

            DFS_ALARM = [dfPQ_CSI[(dfPQ_CSI['CSI_INF_FINAL']<=1) & (dfPQ_CSI['CSI_INF_FINAL']>0)],
                        dfPV_CSI[(dfPV_CSI['CSI_INF_FINAL']<=1) & (dfPV_CSI['CSI_INF_FINAL']>0)],]

            CasosAlarme = pd.concat(DFS_ALARM, axis=0,)
            filtroalarme = (CasosAlarme.index.to_frame()[['Dia', 'Hora']].apply(tuple, axis=1).isin(conjunto_filtros_INSG))
            CasosAlarme = CasosAlarme[~filtroalarme]
            CasosAlarme = CasosAlarme.sort_index()
            lista = [index[:2] for index in CasosAlarme.index]
            conjunto_filtros_ALRM = set(map(tuple, lista))
            filtro1 = (OPF_NC[['Dia', 'Hora']].apply(tuple, axis=1).isin(conjunto_filtros_ALRM))
            OPF_Filtrado = OPF_NC[filtro1]
            filtro2 = (PWF_CV[['Dia', 'Hora']].apply(tuple, axis=1).isin(conjunto_filtros_ALRM))
            PWF_Filtrado = PWF_CV[filtro2]
            print('====================================================================')
            print('Numero de casos com indice  ALARME: ' + str(len(conjunto_filtros_ALRM)) + '=> ' + str(round(len(conjunto_filtros_ALRM)/1344*100,2)))
            print('====================================================================')
            print('Numero de casos com indice ALARME que são não Convergidos no OPF: ' + str(len(OPF_Filtrado)) + '=> ' + str(round(len(OPF_Filtrado)/1344*100,2)))
            
            # ===========================================================================================================================================================
            print('===========================SOBRETENSÃO==============================')
            DFS_INS = [dfPQ_CSI[dfPQ_CSI['CSI_SUP_FINAL']>1],
                    dfPV_CSI[dfPV_CSI['CSI_SUP_FINAL']>1]]
            CasosInseguros = pd.concat(DFS_INS, axis=0,)
            CasosInseguros = CasosInseguros.sort_index()
            lista = [index[:2] for index in CasosInseguros.index]
            conjunto_filtros_INSG = set(map(tuple, lista))
            filtro1 = (OPF_NC[['Dia', 'Hora']].apply(tuple, axis=1).isin(conjunto_filtros_INSG))
            OPF_Filtrado = OPF_NC[filtro1]
            filtro2 = (PWF_CV[['Dia', 'Hora']].apply(tuple, axis=1).isin(conjunto_filtros_INSG))
            PWF_Filtrado = PWF_CV[filtro2]
            print('Numero de casos com indice >  1, INSEGUROS: ' + str(len(conjunto_filtros_INSG)) + '=> ' + str(round(len(conjunto_filtros_INSG)/1344*100,2)))
            print('====================================================================')
            print('Numero de casos com indice >  1 que são não Convergidos no OPF: ' + str(len(OPF_Filtrado)) + '=> ' + str(round(len(OPF_Filtrado)/1344*100,2)) )

            DFS_ALARM = [dfPQ_CSI[(dfPQ_CSI['CSI_SUP_FINAL']<=1) & (dfPQ_CSI['CSI_SUP_FINAL']>0)],
                        dfPV_CSI[(dfPV_CSI['CSI_SUP_FINAL']<=1) & (dfPV_CSI['CSI_SUP_FINAL']>0)],]

            CasosAlarme = pd.concat(DFS_ALARM, axis=0,)
            filtroalarme = (CasosAlarme.index.to_frame()[['Dia', 'Hora']].apply(tuple, axis=1).isin(conjunto_filtros_INSG))
            CasosAlarme = CasosAlarme[~filtroalarme]
            CasosAlarme = CasosAlarme.sort_index()
            lista = [index[:2] for index in CasosAlarme.index]
            conjunto_filtros_ALRM = set(map(tuple, lista))
            filtro1 = (OPF_NC[['Dia', 'Hora']].apply(tuple, axis=1).isin(conjunto_filtros_ALRM))
            OPF_Filtrado = OPF_NC[filtro1]
            filtro2 = (PWF_CV[['Dia', 'Hora']].apply(tuple, axis=1).isin(conjunto_filtros_ALRM))
            PWF_Filtrado = PWF_CV[filtro2]
            print('====================================================================')
            print('Numero de casos com indice  ALARME: ' + str(len(conjunto_filtros_ALRM)) + '=> ' + str(round(len(conjunto_filtros_ALRM)/1344*100,2)))
            print('====================================================================')
            print('Numero de casos com indice ALARME que são não Convergidos no OPF: ' + str(len(OPF_Filtrado)) + '=> ' + str(round(len(OPF_Filtrado)/1344*100,2)))

## c) Graficos DPI por região

In [ ]:
plt.rcParams["font.family"] = "Times New Roman"
paletadcolor = sns.color_palette()

if plotDPI:
    def plot_indice (df_data, eje_y, name, title, INDICE, xlimites=None,ylimites=None, display_ = False, order = False):

        if display_:
            display(df_data[df_data[INDICE]==df_data[INDICE].max()].T)
        
        fig, axs = plt.subplots(nrows=1, figsize=(10, 6), sharex=False)
        colores = [sns.color_palette("Paired")[1], sns.color_palette("Paired")[3], sns.color_palette("Paired")[5],sns.color_palette("Paired")[7],sns.color_palette("Paired")[9]]
        region_map = {'Nordeste':'Northeast', 'Norte':'North', 'Sudeste-Centro-Oeste':'SE-CW', 'Sul':'South','AC-RO':'AC-RO'}
        for idx, regiao in enumerate(['Norte','Nordeste','Sudeste-Centro-Oeste', 'Sul', 'AC-RO']):
        # for idx, regiao in enumerate(['Nordeste']):

            if order:
                data = df_data.loc[:, :, regiao].sort_values(INDICE, ascending=False)[INDICE]
                data_points_per_day = 10
                num_days = 1344*data_points_per_day / 100
                axs.set_xticks([round(i * num_days) for i in range(data_points_per_day+1)])
                axs.set_xticklabels([f'{i*10}' for i in range(data_points_per_day+1)], fontsize=12, rotation=0, ha='center')
            else:
                data = df_data.loc[:, :, regiao][INDICE]
                data_points_per_day = 48
                num_days = len(data) // data_points_per_day
                axs.set_xticks([i * data_points_per_day for i in range(num_days)])
                axs.set_xticklabels([f'{i+1}' for i in range(num_days)], fontsize=18, rotation=0, ha='center')
            
            axs.plot(data.values, color=colores[idx], label=region_map[regiao], lw=2, linestyle='-')
            
        axs.legend(loc='upper right', fontsize=18)
        axs.tick_params(axis='y', labelsize=24)
        axs.tick_params(axis='x', labelsize=18)
        axs.set_xlabel('Percentage of half hours in a month (%)', fontsize=23)
        axs.set_ylabel(eje_y, fontsize=22)
        axs.set_title(title, fontsize=22)
        if xlimites != None:
            axs.set_xlim(xlimites)
        if ylimites != None:
            axs.set_ylim(ylimites)
        axs.grid(True, linestyle='-', linewidth=1.2, alpha=0.4)
        plt.tight_layout()
        nome = cenario + '/Indice/' + name + '.svg'
        plt.savefig(nome)
        nome = cenario + '/Indice/' + name + '.png'
        plt.savefig(nome, bbox_inches = 'tight')
        plt.show()

    plot_indice (dfPQ_CSI, r'$\mathrm{DPI}_\mathrm{PQ}^\mathrm{u}$', 'DPI_(u)_PQ','','CSI_SUP_FINAL', order=True, ylimites=[0, 1])
    plot_indice (dfPQ_CSI, r'$\mathrm{DPI}_\mathrm{PQ}^\mathrm{l}$', 'DPI_(l)_PQ','', 'CSI_INF_FINAL', order=True, ylimites=[0, 1])
    plot_indice (dfPV_CSI, r'$\mathrm{DPI}_\mathrm{PV}^\mathrm{u}$', 'DPI_(u)_PV' ,'','CSI_SUP_FINAL', order=True, ylimites=[0, 1])
    plot_indice (dfPV_CSI, r'$\mathrm{DPI}_\mathrm{PV}^\mathrm{l}$', 'DPI_(l)_PV', '','CSI_INF_FINAL', order=True, ylimites=[0, 1])

In [ ]:
if plotDPI:

    def plot_indice_1 (df_pv, df_pq, eje_y, title, regiao, limites=None, order = True):

        fig, axs = plt.subplots(nrows=1, figsize=(10, 6))
        colores = [paletadcolor[4], paletadcolor[0], paletadcolor[3],paletadcolor[2],paletadcolor[3]]
        for idx, INDICE in enumerate(['CSI_SUP_FINAL', 'CSI_INF_FINAL']):
            if order:
                datapq = df_pq.loc[:, :, regiao][INDICE]
                datapv = df_pv.loc[:, :, regiao][INDICE]
            else:
                datapq = df_pq.loc[:, :, regiao].sort_values(INDICE, ascending=False)[INDICE]
                datapv = df_pv.loc[:, :, regiao].sort_values(INDICE, ascending=False)[INDICE]
                
            axs.plot(datapq.values, color=colores[idx], label='PQ_'+ INDICE[4:7],lw=2, linestyle='-')
            axs.plot(datapv.values, color=colores[idx+2], label='PV_'+ INDICE[4:7],lw=2, linestyle='-')

        axs.legend(loc='best', fontsize=18)
        # # Calculate the number of data points in a day (assuming each day has 48 data points)
        # data_points_per_day = 48
        # # Calculate the number of days based on the length of the data
        # num_days = len(datapq) // data_points_per_day
        # # Set x-axis ticks and labels for each day
        # axs.set_xticks([i * data_points_per_day for i in range(num_days)])
        # axs.set_xticklabels([f'{i+1}' for i in range(num_days)], fontsize=12, rotation=0, ha='right')
        axs.tick_params(axis='y', labelsize=18)
        axs.tick_params(axis='x', labelsize=18)
        axs.set_xlabel('Operating points', fontsize=23)
        axs.set_ylabel(eje_y, fontsize=20)
        axs.set_title(title, fontsize=25)
        if limites != None:
            axs.set_ylim(limites)
        axs.grid(True, linestyle='--', linewidth=1, alpha=0.2)
        plt.tight_layout()
        nome = cenario + '/Indice/' + title + '.svg'
        plt.savefig(nome)
        nome = cenario + '/Indice/' + title + '.png'
        plt.savefig(nome, bbox_inches = 'tight')
        plt.show()

    plot_indice_1 (dfPV_CSI, dfPQ_CSI, 'DPI', 'North','Norte',order=False)#limites=[0,1.3]
    plot_indice_1 (dfPV_CSI, dfPQ_CSI, 'DPI', 'Northeast','Nordeste',order=False )#limites=[0,1.3], limites=[-0.1,2]
    plot_indice_1 (dfPV_CSI, dfPQ_CSI, 'DPI', 'Southeast Center West','Sudeste-Centro-Oeste',order=False)#limites=[0,1.3], limites=[-0.1,9]
    plot_indice_1 (dfPV_CSI, dfPQ_CSI, 'DPI', 'AC-RO','AC-RO',order=False)#limites=[0,1.3]
    plot_indice_1 (dfPV_CSI, dfPQ_CSI, 'DPI', 'South','Sul',order=False)#limites=[0,1.3], limites=[-0.1,1.5]

In [ ]:
if plotDPI:
    def plot_indice_2 (df, eje_y, name ,title, regiao, INDICE, GB, limites=None, order = True):

        fig, axs = plt.subplots(nrows=1, figsize=(10, 6))
        labelG = {'BIO': 'Bio', 'EOL': 'Wind', 'PCH': 'SHP','UFV': 'Solar', 'UHE': 'Hydro','UTE': 'Thermal', 'SIN': 'Synchronous C.'}
        if GB=='Gen_Type':
            colores = {'BIO': paletadcolor[4], 'EOL': paletadcolor[0], 'PCH': paletadcolor[3],'UFV': paletadcolor[2], 'UHE': paletadcolor[5],'UTE': paletadcolor[1], 'SIN': paletadcolor[6]}
        else:
            colores = {230: paletadcolor[4], 345: paletadcolor[0], 440: paletadcolor[3],500: paletadcolor[2], 525: paletadcolor[5], 765: paletadcolor[1]}
        data = df.loc[:, :, regiao]
        Busgroup = np.array(data.reset_index(GB)[GB].unique())
        print(Busgroup)
        for idx, G_bus in enumerate(Busgroup):
            if order:
                data_ = df.loc[:, :, regiao, G_bus][INDICE]
            else:
                data_ = df.loc[:, :, regiao, G_bus].sort_values(INDICE, ascending=False)[INDICE]
            if GB=='Gen_Type':
                label = labelG[G_bus]
            else:
                label = G_bus
            axs.plot(data_.values, color=colores[G_bus], label= label,lw=2)

        axs.legend(loc='best', fontsize=14)
        # Calculate the number of data points in a day (assuming each day has 48 data points)
        data_points_per_day = 48
        # Calculate the number of days based on the length of the data
        num_days = len(data_) // data_points_per_day
        # Set x-axis ticks and labels for each day
        axs.set_xticks([i * data_points_per_day for i in range(num_days)])
        axs.set_xticklabels([f'{i+1}' for i in range(num_days)], fontsize=18, rotation=0, ha='center')
        axs.tick_params(axis='y', labelsize=18)
        # axs.tick_params(axis='x', labelsize=15)
        axs.set_xlabel('Days', fontsize=23)
        axs.set_ylabel(eje_y, fontsize=20)
        axs.set_title(title, fontsize=25)
        if limites != None:
            axs.set_ylim(limites)
        axs.grid(True, linestyle='-', linewidth=1.2, alpha=0.4)
        plt.tight_layout()
        nome = cenario + '/Indice/' + name + '.svg'
        plt.savefig(nome)
        nome = cenario + '/Indice/' + name + '.png'
        plt.savefig(nome, bbox_inches = 'tight')
        plt.show()

    def main_plot_indice_2(dffPQgb, dffPVgb):
        regioes = df_PQ_reg['REG'].unique()
        region_map = {'Nordeste':'Northeast', 'Norte':'North', 'Sudeste-Centro-Oeste':'SE-CW', 'Sul':'South','AC-RO':'AC-RO'}
        for i in regioes:
            Indice = 'CSI_INF'
            plot_indice_2 (dffPQgb, r'$\mathrm{DPI}_\mathrm{PQ}^\mathrm{l}$', 'DPI_(l)_PQ_' + region_map[i], region_map[i] ,i, Indice, 'VBASEKV',limites=[0,2.5])
            plot_indice_2 (dffPVgb, r'$\mathrm{DPI}_\mathrm{PV}^\mathrm{l}$', 'DPI_(l)_PV_' + region_map[i], region_map[i] ,i, Indice, 'Gen_Type', limites=[0,2.5])
            Indice = 'CSI_SUP'
            plot_indice_2 (dffPQgb, r'$\mathrm{DPI}_\mathrm{PQ}^\mathrm{u}$', 'DPI_(u)_PQ_' + region_map[i], region_map[i] ,i, Indice, 'VBASEKV',limites=[0,2.5])
            plot_indice_2 (dffPVgb, r'$\mathrm{DPI}_\mathrm{PV}^\mathrm{u}$', 'DPI_(u)_PV_' + region_map[i], region_map[i] ,i, Indice, 'Gen_Type', limites=[0,2.5])

    main_plot_indice_2(dffPQgb, dffPVgb)

## d) Graficos Barras CRITICAS Boxplot

In [ ]:
if BP_Criticalbuses:

    def boxplot_plot_PB(dff_filtered_PQ, dff_filtered_PV, df_ind, condition):

        dff_PQ =  dff_filtered_PQ.groupby(by=['REG','VBASEKV','BUS_NAME']).agg(Ocurrencies = ('VBASEKV','count'), 
                                                                            MODV_PU = ('MODV_PU', list),
                                                                            MIN = ('MODV_PU', 'min'),
                                                                            MAX = ('MODV_PU', 'max')
                                                                            )

        dff_PV =  dff_filtered_PV.groupby(by=['REG','Gen_Type','BUS_NAME']).agg(Ocurrencies = ('Gen_Type','count'), 
                                                                            MODV_PU = ('MODV_PU', list),
                                                                            MIN = ('MODV_PU', 'min'),
                                                                            MAX = ('MODV_PU', 'max')
                                                                            )
        DF_dfss = [dff_PQ, dff_PV]
        name = ['PQ', 'PV']
        for idx, dff in enumerate(DF_dfss):
            regions = dff.index.get_level_values(0).unique()
            for Region in regions:
                nt = dff.loc[Region].index.get_level_values(0).unique()
                for vb in nt:
                    df_boxplot = dff.loc[Region,vb].sort_values(['Ocurrencies'], ascending = False)[:15]
                    minimo  = df_boxplot['MIN'].min() - 0.01
                    maximo =  df_boxplot['MAX'].max() + 0.01
                    numbuses =  condition + ' ' + Region + ' - ' + df_ind  + ' - ' +  name[idx] + ' - ' +  str(vb)  + ' - Buses with voltage problems = ' + str(dff.loc[Region,vb].shape[0]) 
                    plot_boxplot(df_boxplot['MODV_PU'], df_boxplot.index, numbuses , 'BUSES', 'VOLTAGE (pu)', text = True, rotation=45, limites=[minimo,maximo])
                    
    def boxplot_problematic_buses(df_busPQ,df_busPV):
        
        dicIndice = ['IndiceInf','IndiceSup']
        for df_ind in dicIndice:

            dff_filtered_PQ = df_busPQ[df_busPQ[df_ind]>1].sort_values(by=['REG','VBASEKV', 'BUS_NAME', 'MODV_PU'], ascending=[True, True, False, True])
            dff_filtered_PV = df_busPV[df_busPV[df_ind]>1].sort_values(by=['REG','Gen_Type', 'BUS_NAME', 'MODV_PU'], ascending=[True, True, False, True])
            boxplot_plot_PB(dff_filtered_PQ, dff_filtered_PV, df_ind, 'Inseguro')

            dff_filtered_PQ = df_busPQ[(df_busPQ[df_ind]<=1) & (df_busPQ[df_ind]>0)].sort_values(by=['REG','VBASEKV', 'BUS_NAME', 'MODV_PU'], ascending=[True, True, False, True])
            dff_filtered_PV = df_busPV[(df_busPV[df_ind]<=1) & (df_busPV[df_ind]>0)].sort_values(by=['REG','Gen_Type', 'BUS_NAME', 'MODV_PU'], ascending=[True, True, False, True])
            boxplot_plot_PB(dff_filtered_PQ, dff_filtered_PV, df_ind, 'Alarme')

    boxplot_problematic_buses(df_busPQ_mod,df_busPV_mod)

## e) Criação pastas e arquivos para simular no Organon

In [ ]:
if resumoIndice:
    df_busPQ_mod['BUS_ID'] = df_busPQ_mod['BUS_ID'].astype(int)
    df_busPV_mod['BUS_ID'] = df_busPV_mod['BUS_ID'].astype(int)
    df_busPV_mod[(df_busPV_mod['IndiceInf']>0.20)].groupby(by = ['REG'])['BUS_ID'].nunique().to_csv(cenario + '/Critical_infPVbuses.txt', header=True, index=True)
    df_busPQ_mod[(df_busPQ_mod['IndiceInf']>0.20)].groupby(by = ['REG'])['BUS_ID'].nunique().to_csv(cenario + '/Critical_infPQbuses.txt', header=True, index=True)
    df_busPV_mod[(df_busPV_mod['IndiceInf']>0.20)].groupby(by = ['REG'])['BUS_ID'].unique().to_csv(cenario + '/Critical_infPVbuses_bus.txt', header=True, index=True)
    df_busPQ_mod[(df_busPQ_mod['IndiceInf']>0.20)].groupby(by = ['REG'])['BUS_ID'].unique().to_csv(cenario + '/Critical_infPQbuses_bus.txt', header=True, index=True)

In [ ]:
# dff_filtered_PV = df_busPV_mod[(df_busPV_mod['IndiceInf']>0.20)].sort_values(by=['Gen_Type'])
# df_counted = dff_filtered_PV[['BUS_ID', 'BUS_NAME']].value_counts().reset_index()
# df = df_counted
# num_rows = ((len(df) + 29) // 30)//2
# num_columns = 2

# if len(df)>30:
#     fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, 6 * num_rows), sharex=False)
#     axes = axes.flatten()
#     # Iterate over subplots and plot each segment
#     for i, ax in enumerate(axes):
#         start_idx = i * 30
#         end_idx = min((i + 1) * 30, len(df))
        
#         # Plot the bar chart for the current segment
#         df_segment = df.iloc[start_idx:end_idx]
#         ax.bar(df_segment['BUS_NAME'], df_segment[0])
        
#         # Set labels and title
#         ax.set_ylabel('Occurrences', fontsize=12)
#         ax.set_title(f'Segment {i + 1}')
#         ax.set_xticklabels(df_segment['BUS_NAME'], rotation=90, ha='center', fontsize=10)
#     # Remove empty subplots
#     for i in range(len(df), len(axes)):
#         fig.delaxes(axes[i])

#     # Adjust layout for better appearance
#     fig.suptitle('Critical Buses = ' + str(len(df)) , fontsize=16, y=1)
# else:
#     fig, ax = plt.subplots(1, 1, figsize=(10, 6 ), sharex=False)
#     ax.bar(df['BUS_NAME'], df[0])
#     ax.set_ylabel('Occurrences', fontsize=12)
#     ax.set_xticklabels(df['BUS_NAME'], rotation=90, ha='center', fontsize=10)

# plt.tight_layout()
# plt.savefig(cenario+'/Fig_CriticalBuses_all.png')
# plt.show()

In [ ]:
## dff_filtered_PQ = df_busPQ_mod[(df_busPQ_mod['IndiceInf']>0.8)].sort_values(by=['Dia','Hora', 'BUS_ID'], ascending=[True, True, True])
## barras = list(dff_filtered_PQ['BUS_ID'].unique()) + list(dff_filtered_PV['BUS_ID'].unique())

# nome = path_folder.split('/')[-2]+'_mud_01_'
# dff_filtered_PV = df_busPV_mod[(df_busPV_mod['IndiceInf']>0.20)].sort_values(by=['Gen_Type'])
# # dff_filtered_PV = DF_Merged02.sort_values(by=['Gen_Type'])
# EOL_dff_filtered_PV = dff_filtered_PV[dff_filtered_PV['Gen_Type'] == 'EOL']['BUS_ID'].unique()
# barras = list(dff_filtered_PV['BUS_ID'].unique())
# veclines = []
# for bus in barras:
#     barra = str(int(bus))
#     if bus in EOL_dff_filtered_PV:
#         veclines.append('VOLT    ' + barra+'  1.    0.950   1.100\n')
#     else:
#         veclines.append('VOLT    ' + barra+'  1.    0.950   1.100\n')

# path1 = 'C:/Users/david/OneDrive/Documents/FERV_BAIXADOS/OPF SIMULADOS/TENSAO_FPO.opf'
# with open(path1, 'r') as file:
#     lines = file.readlines()

# lines[8:8] = veclines
# path2 = 'C:/Users/david/OneDrive/Documents/FERV_BAIXADOS/OPF SIMULADOS/TENSAO_FPO_novo.opf'
# with open(path2, 'w') as file:
#     list_as_string = ''.join(map(str, lines))
#     file.write(list_as_string)

#====================================================================================================================================================================================
# opfNC = Opf_Simulation(path_folder,nome) # Para gerar as pastas 
#====================================================================================================================================================================================

In [ ]:
# from Opf_Simulation_filtrado import *
# df = Df_PV_UV[Df_PV_UV['UV condition'] == 'Inseguro'].reset_index()
# nome = path_folder.split('/')[-2]+'_mud_GER_'
# Opf_Simulation_filt(path_folder,nome, df)

# Analise Indice Violação de Tensão

In [ ]:
if ComputeDPI:
    if resumoIndice:
        def discriminarIndice2(x):
            if x>1:
                return 'Inseguro'
                # return 3
            elif (x<=1) & (x>0):
                return 'Alarme'
                # return 2
            elif x == 0:
                return 'Seguro'
                # return 1

        Df_IndiceT2 = pd.concat([dfPQ_CSI[['CSI_SUP_FINAL','CSI_INF_FINAL']],dfPV_CSI[['CSI_SUP_FINAL','CSI_INF_FINAL']]], axis=0, keys=['DPI_PQ', 'DPI_PV'])
        Df_IndiceT2['OV condition'] = Df_IndiceT2['CSI_SUP_FINAL'].apply(lambda x: discriminarIndice2(x))
        Df_IndiceT2['UV condition'] = Df_IndiceT2['CSI_INF_FINAL'].apply(lambda x: discriminarIndice2(x))
        Df_IndiceT2.rename(columns={'CSI_SUP_FINAL':'OV DPI','CSI_INF_FINAL':'UV DPI'}, inplace=True)

        # dataNE_1 = Df_IndiceT2.loc[:,:,:,'Nordeste']
        # dataN_1 = Df_IndiceT2.loc[:,:,:,'Norte']
        # dataSECO_1 = Df_IndiceT2.loc[:,:,:,'Sudeste-Centro-Oeste']
        # dataACRO_1 = Df_IndiceT2.loc[:,:,:,'AC-RO']
        # dataSul_1 = Df_IndiceT2.loc[:,:,:,'Sul']

        Df_seguros_PQ = Df_IndiceT2.loc['DPI_PQ'][((Df_IndiceT2.loc['DPI_PQ']['OV DPI']==0) & (Df_IndiceT2.loc['DPI_PQ']['UV DPI']==0))][['OV condition', 'OV DPI']]
        Df_seguros_PV = Df_IndiceT2.loc['DPI_PV'][((Df_IndiceT2.loc['DPI_PV']['OV DPI']==0) & (Df_IndiceT2.loc['DPI_PV']['UV DPI']==0))][['OV condition', 'OV DPI']]
        Df_PQ_OV = Df_IndiceT2.loc['DPI_PQ'][~((Df_IndiceT2.loc['DPI_PQ']['OV DPI']==0) & (Df_IndiceT2.loc['DPI_PQ']['UV DPI']>0))].sort_values('OV DPI', ascending=False)[['OV condition', 'OV DPI']]
        Df_PQ_UV = Df_IndiceT2.loc['DPI_PQ'][~((Df_IndiceT2.loc['DPI_PQ']['UV DPI']==0) & (Df_IndiceT2.loc['DPI_PQ']['OV DPI']>0))].sort_values('UV DPI', ascending=False)[['UV condition', 'UV DPI']]
        Df_PV_OV = Df_IndiceT2.loc['DPI_PV'][~((Df_IndiceT2.loc['DPI_PV']['OV DPI']==0) & (Df_IndiceT2.loc['DPI_PV']['UV DPI']>0))].sort_values('OV DPI', ascending=False)[['OV condition', 'OV DPI']]
        Df_PV_UV = Df_IndiceT2.loc['DPI_PV'][~((Df_IndiceT2.loc['DPI_PV']['UV DPI']==0) & (Df_IndiceT2.loc['DPI_PV']['OV DPI']>0))].sort_values('UV DPI', ascending=False)[['UV condition', 'UV DPI']]
        Df_IndiceT2.to_csv(cenario+'/Indice.csv')
        Df_PV_OV.to_csv(cenario+'/IndicePV_OV.csv')
        Df_PV_UV.to_csv(cenario+'/IndicePV_UV.csv')
        Df_PQ_OV.to_csv(cenario+'/IndicePQ_OV.csv')
        Df_PQ_UV.to_csv(cenario+'/IndicePQ_UV.csv')

        path_script_org = cenario + "/RelatorioIndice.txt"
        numeroPO = len(set(Df_IndiceT2.index.to_frame()[['Dia','Hora']].apply(tuple, axis=1).values))
        with open(path_script_org, 'w') as f:
            f.write('O numero de pontos de operação analisados são: ' + str(numeroPO) + '\n')
            f.write('=============================\n Informação Barras PQ:\n=============================\n')
            regions = Df_PQ_OV.reset_index('REG')['REG'].unique()
            for reg in regions:
                f.write('- Sobretensão ' + reg +'\n')
                df_reg_sob= Df_PQ_OV.loc[:,:,reg]
                f.write('numero de casos Inseguros: '+ str(df_reg_sob[df_reg_sob['OV condition']=='Inseguro'].shape[0])+'\n')
                f.write('numero de casos Alarme: '+ str(df_reg_sob[df_reg_sob['OV condition']=='Alarme'].shape[0])+'\n')
                f.write('numero de casos Seguros: '+ str(df_reg_sob[df_reg_sob['OV condition']=='Seguro'].shape[0])+'\n')
                f.write('- Subtensão '+ reg +'\n')
                df_reg_sub = Df_PQ_UV.loc[:,:,reg]
                f.write('numero de casos Inseguros: ' + str(df_reg_sub[df_reg_sub['UV condition']=='Inseguro'].shape[0])+'\n')
                f.write('numero de casos Alarme: ' + str(df_reg_sub[df_reg_sub['UV condition']=='Alarme'].shape[0])+'\n')
                f.write('numero de casos Seguros: ' + str(df_reg_sub[df_reg_sub['UV condition']=='Seguro'].shape[0])+'\n')
                f.write('--------------------------\n')
            f.write('=============================\n Informação Barras PV:\n=============================\n')
            for reg in regions:
                f.write('- Sobretensão ' + reg +'\n')
                df_reg_sob= Df_PV_OV.loc[:,:,reg]
                f.write('numero de casos Inseguros: '+ str(df_reg_sob[df_reg_sob['OV condition']=='Inseguro'].shape[0])+'\n')
                f.write('numero de casos Alarme: '+ str(df_reg_sob[df_reg_sob['OV condition']=='Alarme'].shape[0])+'\n')
                f.write('numero de casos Seguros: '+ str(df_reg_sob[df_reg_sob['OV condition']=='Seguro'].shape[0])+'\n')
                f.write('- Subtensão '+ reg +'\n')
                df_reg_sub = Df_PV_UV.loc[:,:,reg]
                f.write('numero de casos Inseguros: ' + str(df_reg_sub[df_reg_sub['UV condition']=='Inseguro'].shape[0])+'\n')
                f.write('numero de casos Alarme: ' + str(df_reg_sub[df_reg_sub['UV condition']=='Alarme'].shape[0])+'\n')
                f.write('numero de casos Seguros: ' + str(df_reg_sub[df_reg_sub['UV condition']=='Seguro'].shape[0])+'\n')
                f.write('--------------------------\n')

### a) ANALISE CORRELAÇÃO

In [ ]:
# if CorrelationAnalise:
#     concatenated_HVDC = pd.concat([df_HVDC.loc['Elo_FOZ-IBIUNA'][' P(MW)'], df_HVDC.loc['Elo_PVEL-ARARQ'][' P(MW)'], df_HVDC.loc['Elo_XINGU-SE'][' P(MW)']], axis=1,keys=['Elo_FOZ-IBIUNA', 'Elo_PVEL-ARARQ', 'Elo_XINGU-SE'])
#     concatenated_FluxoAC = pd.concat([DF_Intercambios.loc['Fluxo_N-S']['MW:From-To'],DF_Intercambios.loc['Fluxo_NE-N']['MW:From-To'], DF_Intercambios.loc['Fluxo_NE-SE']['MW:From-To'], DF_Intercambios.loc['Fluxo_SUL-SECO']['MW:From-To']], axis=1, keys=['Fluxo_N-S', 'Fluxo_NE-N', 'Fluxo_NE-SE', 'Fluxo_SUL-SECO'])
#     def merge_datapairplot(DF_Intercambios2,DF_REGIONAL_GER,df_HVDC_2, region, col_intercambios_AC):
        
#         merged_df1 = DF_Intercambios2[col_intercambios_AC].merge(DF_REGIONAL_GER.loc[:, :, region][['PG_MW','QG_MVAR', 'PG/PL', 'QG/QL', 'PG_FERV', 'QG_EOL']], on=['Dia', 'Hora'])
#         final_merged_df = merged_df1.merge(df_HVDC_2, on=['Dia', 'Hora'])

#         return final_merged_df

#     Df__SECO = merge_datapairplot(concatenated_FluxoAC, DF_REGIONAL_GER, concatenated_HVDC, 'Sudeste-Centro-Oeste',['Fluxo_N-S', 'Fluxo_NE-SE', 'Fluxo_SUL-SECO'])
#     Df__N = merge_datapairplot(concatenated_FluxoAC, DF_REGIONAL_GER, concatenated_HVDC['Elo_XINGU-SE'], 'Norte',['Fluxo_N-S', 'Fluxo_NE-N'])
#     Df__NE = merge_datapairplot( concatenated_FluxoAC,DF_REGIONAL_GER, concatenated_HVDC['Elo_XINGU-SE'], 'Nordeste',['Fluxo_NE-N', 'Fluxo_NE-SE'])
#     Df__SUL = merge_datapairplot(concatenated_FluxoAC, DF_REGIONAL_GER, concatenated_HVDC['Elo_FOZ-IBIUNA'], 'Sul',['Fluxo_SUL-SECO'])
#     Df__ACRO = merge_datapairplot(concatenated_FluxoAC, DF_REGIONAL_GER, concatenated_HVDC['Elo_PVEL-ARARQ'], 'AC-RO',['Fluxo_NE-SE','Fluxo_N-S', 'Fluxo_SUL-SECO'])

#     dfindices = [Df_PQ_OV,Df_PQ_UV,Df_PV_OV,Df_PV_UV]
#     typebus = ['PQ','PQ','PV','PV']
#     lista_reg = [Df__SECO,Df__N,Df__NE,Df__SUL,Df__ACRO]
#     reg = ['Sudeste-Centro-Oeste','Norte','Nordeste','Sul','AC-RO']
#     label_colors = {'Inseguro': sns.color_palette("Paired", 10)[5], 'Seguro': sns.color_palette("Paired", 10)[3],'Alarme': sns.color_palette("Paired", 10)[0], }
#     # mpl.rcParams["axes.labelsize"] = 12
#     for idx, dfreg in enumerate(lista_reg):
#         for idx2, df in enumerate(dfindices):
#             label = df.columns[0]
#             merged_df = df.loc[:,:,reg[idx]].merge(dfreg, on=['Dia', 'Hora']).sort_values(label)

#             for columns in merged_df.columns[1:]:
#                 fig, axs = plt.subplots(nrows=1, figsize=(10, 6))
#                 sns.histplot(data=merged_df, x=columns, hue=label, palette=label_colors, alpha=0.7, element="step", stat="density", common_norm=False)
#                 plt.title(reg[idx] + '-' + typebus[idx2])
#                 plt.ylabel("Density", fontsize=15)
#                 plt.xlabel(columns.replace('_',' '), fontsize=15)
#                 name = str(label +'_' + reg[idx] +'_'+ typebus[idx2] +'_'+ columns).replace('/','-')
#                 nome = cenario + '/Analise Histograma/' + name +'.png'
#                 plt.savefig(nome)
#                 plt.show()
                
#             # g = sns.PairGrid(merged_df, hue=label, corner=True, palette=label_colors)
#             # g.map_diag(sns.histplot, multiple="stack",element="step")
#             # g.map_offdiag(plt.scatter)
#             # g.add_legend(fontsize= '20')
#             # name = str(reg[idx] +' '+ typebus[idx2] +' '+ label).replace('_',' ').replace('-',' ')
#             # nome = cenario + '/Analise Correlação/' + name +'.png'
#             # plt.savefig(nome)
#             # plt.show()

### b) Leitura do arquivo Tensão.opf

In [ ]:
# # Script para ler o arquivo TENSAO_FPO.opf
# # *************************************************************************************
# path = 'C:/Users/David/OneDrive/Desktop/opfFolder/TENSAO_FPO.opf'
# with open(path, 'r') as file:
#     lines = file.readlines()

# data = []
# # Iterar pelas linhas do arquivo
# for i in range(len(lines)):
#     if (i>7) & (i<451):
#         line = lines[i].strip().split()
#         data.append(line)

# df = pd.DataFrame(data)
# df[1] = df[1].astype(float)

# busdf = Df_VF[(Df_VF['Dia'] == '01') & (Df_VF['Hora'] == '00-00')]
# dff = busdf[busdf['BUS_ID'].isin(df[1].values)]
# display(dff['VBASEKV'].value_counts())
# display(dff[dff['Gen_Type'].isin(['PCH', 'BIO'])][['BUS_ID', 'BUS_NAME', 'MODV_PU', 'Gen_Type', 'REG', 'VBASEKV', 'TP', 'BASE_MVA', 'PG_MW', 'QG_MVAR', 'PMAX_MW','PMIN_MW', 'QMX_MVAR', 'QMN_MVAR', 'Ger_Units']])

# # Step 3: Identify the values from the second DataFrame for filtering
# values_to_exclude = dff[dff['VBASEKV']==500]['BUS_ID'].tolist()
# # Step 4: Filter the rows of the first DataFr
# df1_filtered = df[df[1].isin(values_to_exclude)]
# # Step 5: Save the filtered DataFrame to a new text file
# df1_filtered.to_csv('filtered_text_file.txt', sep='\t', index=False)

In [ ]:
# dff_filtered_PV = df_busPV_mod[(df_busPV_mod['IndiceInf']>0.8)].sort_values(by=['Dia','Hora', 'BUS_ID'], ascending=[True, True, True])
# veclines = []
# for bus in dff_filtered_PV['BUS_ID'].unique():
#     barra = str(int(bus))
#     veclines.append('VOLT    ' + barra+'  1.    0.950  1.052\n')
# # *************************************************************************************
# path1 = 'C:/Users/David/OneDrive/Desktop/opfFolder/TENSAO_FPO.opf'
# with open(path1, 'r') as file:
#     lines = file.readlines()

# lines[8:8] = veclines
# path2 = 'C:/Users/David/OneDrive/Desktop/opfFolder/TENSAO_FPO_novo.opf'
# with open(path2, 'w') as file:
#     list_as_string = ''.join(map(str, lines))
#     file.write(list_as_string)